In [41]:
import os
import numpy as np
import math
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras import models, layers, optimizers
import tensorflow as tf

In [3]:
truedirpath = 'Folds/3folds/fold1/Train/True'
truedir = os.listdir(truedirpath)
truedir = np.array([int(i[:-4]) for i in truedir])

In [4]:
falsedirpath = 'Folds/3folds/fold1/Train/False'
falsedir = os.listdir(falsedirpath)
falsedir = np.array([int(i[:-4]) for i in falsedir])

In [5]:
totalcount = truedir.shape[0]+falsedir.shape[0]

In [17]:
bs = 32

In [18]:
numberofiterations = math.ceil((truedir.shape[0] + falsedir.shape[0])/bs)

In [23]:
from Models.models import dn121scratch as dns

In [42]:
model = dns()
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
# my_callbacks = [
#     tf.keras.callbacks.ModelCheckpoint(
#         filepath='checkpoints2/'+'fold_{}____'.format(f)+datetime.now().strftime("%d_%m_%Y____%H_%M_%S")+'.h5',
#         monitor = 'val_loss',
#         save_best_only = True
#      ),
# ]

In [48]:
imagesleft = totalcount
for i in range(numberofiterations):
    
    jcount = min(imagesleft, bs)
    true_batchcount = round(truedir.shape[0]/(truedir.shape[0]+falsedir.shape[0])*jcount)
    false_batchcount = jcount - true_batchcount
    truebatch = np.zeros([true_batchcount,600,600,3])
    falsebatch = np.zeros([false_batchcount,600,600,3])
    trueims = truedir[i*true_batchcount:(i+1)*true_batchcount]
    falseims = falsedir[i*false_batchcount:(i+1)*false_batchcount]
 
#     print(trueims.shape[0], falseims.shape[0])
   
    for j in range(trueims.shape[0]):
        truebatch[j] = img_to_array(load_img(truedirpath+'/'+str(trueims[j])+'.tif'))
    
    for j in range(falseims.shape[0]):
        falsebatch[j] = img_to_array(load_img(falsedirpath+'/'+str(falseims[j])+'.tif'))
        
    batch = np.concatenate([truebatch, falsebatch], axis = 0)
    y = np.concatenate([np.zeros([truebatch.shape[0]]), np.ones([falsebatch.shape[0]])], axis = 0)
    
    permindices = np.random.permutation(batch.shape[0])
    batch = batch[permindices]
    y = y[permindices].reshape(-1,1)
    print(y.shape)
    
    del trueims;del falseims;del truebatch;del falsebatch
    imagesleft-=bs
    
    model.fit(
        x = batch,
        y = y,
        epochs = 1,
        verbose = 2,
    )
    
    del batch;del y;
    

(32, 1)
1/1 - 0s - loss: 0.4489 - accuracy: 0.7500
(32, 1)
1/1 - 0s - loss: 0.8901 - accuracy: 0.5625
(32, 1)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.preprocessing.image import array_to_img
plt.imshow(array_to_img(batch[5]))

In [34]:
a = np.array([7,14,21,28,35,49,56,63,70])
b = np.random.permutation(9)
a = a[b]

print(b)
print(a)

[3 6 8 0 2 5 1 4 7]
[28 56 70  7 21 49 14 35 63]
